<a href="https://colab.research.google.com/github/Swayamprakashpatel/PU_DL/blob/main/Block_Wise_Image_Scanning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from sklearn.utils import shuffle
%matplotlib inline

# Function to extract features from an image using a sliding window
def extract_features(image, window_size, stride):
    features = []
    image_height, image_width, _ = image.shape

    for y in range(0, image_height - window_size[0] + 1, stride):
        for x in range(0, image_width - window_size[1] + 1, stride):
            window = image[y:y+window_size[0], x:x+window_size[1]]
            # Process the window to extract features (you can modify this)
            # For simplicity, we'll flatten the window as a feature
            features.append(window.flatten())

    return np.array(features)

# Set Batch Size and Image Height and Image Width
batch_size = 10
IMG_HEIGHT, IMG_WIDTH = (100, 100)
WINDOW_SIZE = (50, 50)  # Adjust this as needed
STRIDE = 25  # Adjust this as needed

# Define the CNN model
model = Sequential([
    Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), padding='same', activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(256, (3,3), padding='same', activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Model Compilation
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Directory paths for your dataset
train_dir = 'path_to_your_training_data_directory'
validation_dir = 'path_to_your_validation_data_directory'

# ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of augmented data from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='binary'
)

# Train the model
hist = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=5,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size
)

# Accuracy Print
train_acc = max(hist.history['accuracy'])
val_acc = max(hist.history['val_accuracy'])
train_loss = min(hist.history['loss'])
val_loss = min(hist.history['val_loss'])
print('Training accuracy is', train_acc)
print('Validation accuracy is', val_acc)
print('Training loss is', train_loss)
print('Validation loss is', val_loss)
